In [ ]:
import urllib
import urllib.request
import urllib.parse
import bs4
import re
import os
import time
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import konlpy.tag
import nltk
import json
#import pandas as pd
#import numpy as np
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
#import re

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('merged_comments.csv')

In [ ]:
scores = tuple(df.iloc[:,3])
texts = tuple(df.iloc[:,4])

In [ ]:
filtered_texts = []
filtered_labels = []

for text, score in zip(texts, scores):
    if 4 <= score <= 8:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 4 -> 부정, -1
    # 8 ~ 10 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 8 else -1)

In [ ]:
# You can also use the following method
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)

In [ ]:
def tfidf_extractor(corpus):
    # returns a tf-idf based DTM
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [ ]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)

In [ ]:
%%time
lr = LogisticRegression(C=0.1, penalty='l1', random_state=0) # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

In [ ]:
%%time
MNB=MultinomialNB()
MNB.fit(train_tfidf_features, train_labels) 
pred_labels = MNB.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFT = RandomForestClassifier(n_estimators =100)

In [ ]:
RFT.fit(train_tfidf_features, train_labels) 

In [ ]:
pred_labels = RFT.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

In [ ]:
komoran = Komoran()
def tokenizer_noun(doc):
    return komoran.nouns(doc)

def tokenizer_morphs(doc):
    return komoran.morphs(doc)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=tokenizer_morphs)

In [ ]:
multi_nbc = Pipeline([('vect', tfidf), ('nbc', MultinomialNB())])

In [ ]:
#lr = LogisticRegression(C=0.1, penalty='l1', random_state=0) # Lasso regression
multi_nbc.fit(train_tfidf_features, train_labels) # 학습
#pred_labels = lr.predict(test_tfidf_features)
#print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
#print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

In [ ]:
df_movie = df.iloc[:,3:5]

In [ ]:
df_movie.columns = ['rating','review']

In [ ]:
df_movie.head()
df_movie.shape

In [ ]:
emotion_class = [
    "POS" if df_movie.iloc[i]['rating'] >= 8
    else
    "NEU" if df_movie.iloc[i]['rating'] >= 4
    else
    "NEG"
    for i in range(df_movie.shape[0])
]
df_movie["class"] = emotion_class

In [ ]:
df_movie.to_pickle("./df_movie_review.pkl")

In [ ]:
df_movie = pd.read_pickle("./df_movie_review.pkl")

In [ ]:
twitter = Twitter()
def tokenizer_twitter_morphs(doc):
    return twitter.morphs(doc)

def tokenizer_twitter_noun(doc):
    return twitter.nouns(doc)

def tokenizer_twitter_pos(doc):
    return twitter.pos(doc, norm=True, stem=True)

In [ ]:
komoran = Komoran()
def tokenizer_noun(doc):
    return komoran.nouns(doc)

def tokenizer_morphs(doc):
    return komoran.morphs(doc)


In [ ]:
df_movie['token_review'] = df_movie['review'].apply(tokenizer_twitter_morphs)

In [ ]:
tokens = [ t for d in df_movie['token_review'] for t in d]

In [ ]:
text = nltk.Text(tokens, name='NMSC')

In [ ]:
print(text.vocab().most_common(10))

In [ ]:
print(text.vocab().most_common()[:-20:-1])

In [ ]:
#import seaborn as sns

In [ ]:
#sns.set_style("darkgrid")
#sns.set_palette('hls')

In [ ]:
#text.plot(20)

In [ ]:
#text.concordance('영화')

In [ ]:
X_train = df_movie.loc[:, 'review'].values
y_train = df_movie.loc[:, 'class'].values

In [ ]:
tfidf = TfidfVectorizer(tokenizer=tokenizer_morphs)

In [ ]:
multi_nbc = Pipeline([('vect', tfidf), ('nbc', MultinomialNB())])

In [ ]:
sgd_clf = Pipeline([('vect', tfidf), ('sgd', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42))])

In [ ]:
start = time()
sgd_clf.fit(X_train, y_train)
end = time()
print('Time: {:f}s'.format(end-start))

In [ ]:
len(testset)

In [ ]:
y_pred = sgd_clf.predict(testset)
#print("테스트 정확도: {:.3f}".format(accuracy_score(y_train, y_pred)))

In [ ]:
tmp=pd.DataFrame(testset)
tmp['y_pred']=y_pred

In [ ]:
print(tmp[tmp['y_pred']!="POS"][0])#[17])

In [ ]:
from twython import Twython

APP_KEY='9YZNN1Nec1BnHApzLseYtEIi8' 

APP_SECRET='ktUDazXe5Qmkn8OFgKs8KO9KIWycIuCxD1kPCRF8RwEkVuaE3Z'

OAUTH_TOKEN='1061859779008557056-yRZX6t2HSJyQmuaFY0CpuJbkaybSpu'

OAUTH_TOKEN_SECRET='ifIbGQDOiIX3To6tX3IRCdKdb3pALwh2gHEgLCG7L0aCL'

twitterr = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [ ]:
tweets = twitterr.search(q='보헤미안 랩소디', count=200)
data = [(t['user']['screen_name'], t['text'], t['created_at']) for t in tweets['statuses']]

In [ ]:
?twitterr.search(q='클레', count=200)

In [ ]:
data2 = [t['text']  for t in tweets['statuses']]
tary=np.array(data2)
step1 =list(map(lambda x : re.sub(r'[^\s\d\w\.\?\!\,]','',x),tary))
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 

result = list(map(lambda x : hangul.sub('', x),step1))
testset = np.array(result)

In [ ]:
len(testset)

In [ ]:
#data22=np.array(data2).apply(lambda x : re.sub(r'[^\s\d\w\.\?\!\,]','', x['title']),axis =1)

In [ ]:
start = time()
multi_nbc.fit(X_train, y_train)
end = time()
print('Time: {:f}s'.format(end-start))

In [ ]:
y_pred = multi_nbc.predict(X_train)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_train, y_pred)))

In [ ]:
out = pd.DataFrame()

In [ ]:
out['X_train']=X_train
out['y_pred']=y_pred
out['y_train']=y_train

In [ ]:
out[y_pred=='NEG']


In [ ]:
a[a['y_pred']=='NEU']